In [1]:
import numpy as np
import time 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
from jupyterthemes import jtplot

In [ ]:
'''
Inputs:
max_depth, rho, CN_lambda, qtz_rho (rho)--density, total_time, alpha (rock_thermal_diff), MAT,
AMP (Temp. amplitude), Be10_P0, C14_P0, scaling_factor
'''

In [58]:
# total_time = 25000
# time_ER_shift = 0
# SLHL_C14 = 14.1
# SLHL_Be10 = 4.01
# scaling_factor = 1
# save_output = True
# save_only_surf = True

In [3]:
# max_depth = 1500
# dat_lin = np.arange(0,max_depth,1)
# rock_rho = 2.7
# CN_lambda = 160
# mu = rock_rho / CN_lambda
# dt = 1

dat_lin = np.arange(0,max_depth[1] + 1,1)

In [13]:
def C14fxn(dt, depth, conc0, scaling_factor):
       
    C14_P0 = SLHL_C14 * scaling_factor   # production rate atoms / g / yr
    thalf = 5730    # half life 14-C, years
    decay = np.log(2) / thalf
    f_tot = 1.72*10**(-2)

    mu = 2.62/160 # g * cm^-2
    A1 = 170.6 # Granger and Smith (2000)
    A2 = 36.75 # ibid.
    L1 = 788.6 # g cm^-2; Granger and Smith (2000)
    L2 = 2688 # g cm*-2; ibid.
    
    muons = [f_tot * (A1*np.e**((-rock_rho * depth[i])/L1) + A2*np.e**((-2.62*depth[i])/L2)) for i in depth]  
    spallation = [C14_P0 * np.e**(-mu * depth[i]) for i in depth]
    production_tot_14 = [spallation[i] + muons[i] for i in depth]
    
    conc14C = [conc0[i]*np.e**(-decay) + (production_tot_14[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]

    return conc14C

In [29]:
def Be10fxn(dt, depth, conc0, scaling_factor):
    
    Be10_P0 = SLHL_Be10 * scaling_factor
    decay = 4.998e-7    
    B_Be = 0.026 # Granger and Smith 2000
    L3 = 4360 # g cm^-2 Granger and Smith 2000
    
    muons_10 = [0.026 * np.e**(-rock_rho * depth[i] / L3) for i in dat_lin]
    spallogenic_10 = [Be10_P0 * np.e**(-mu * depth[i]) for i in dat_lin]
    production_tot_10 = [muons_10[i] + spallogenic_10[i] for i in dat_lin]
    
    conc10Be = [conc0[i] * np.e**(-decay) + (production_tot_10[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]    

    return conc10Be

In [32]:
def CRN_fxn(conc10Be, conc14C):
    
    conc10Be = Be10fxn(dt, dat_lin, conc10Be, scaling_factor)
    conc14C = C14fxn(dt, dat_lin, conc14C, scaling_factor)    

    first_Befxn = interp1d(x = dat_lin, y = conc10Be)
    first_Cfxn = interp1d(x = dat_lin, y = conc14C)
    C_Be_ratio = [conc14C[i]/conc10Be[i] for i in range(len(conc10Be))]

    return conc10Be, conc14C, C_Be_ratio

In [56]:
def CRN_loop_fxn(total_time, time_ER_shift):

        conc14C = [0] * len(dat_lin)
        conc10Be = [0] * len(dat_lin)
        C_Be_surf = np.empty((total_time,))
        C_surf = np.empty((total_time,))
        Be_surf = np.empty((total_time,))
        C_tot = np.empty((len(dat_lin),total_time))
        Be_tot = np.empty((len(dat_lin),total_time))

        start_time = time.time()

        for i in range(0, total_time):
            conc10Be, conc14C, C_Be_ratio = CRN_fxn(conc10Be, conc14C)  
            C_tot[:,i] = conc14C
            Be_tot[:,i] = conc10Be
            C_Be_surf[i,] = np.average(C_Be_ratio[0:6])
            C_surf[i,] = np.average(conc14C[0:6])
            Be_surf[i,] = np.average(conc10Be[0:6])

        ratios = [(C_tot[0,i] / Be_tot[0,i]) for i in range(total_time)]
        
        print("--- %s seconds ---" % (time.time() - start_time))
        
        if save_output == True:                
            export_mat = np.empty((total_time,4))
            export_mat[:,0] = C_surf
            export_mat[:,1] = Be_surf
            export_mat[:,2] = ratios
            export_mat[0,3] = 0.0
            
            C_export = C_tot
            Be_export = Be_tot
                
            if save_only_surf == True:   
                np.savetxt('NE_C14_10Be_ER0.0_expmat.csv',
                           export_mat,
                           delimiter = ',')
            else:
                np.savetxt('NE_C14_10Be_ER0.0_expmat.csv',
                           export_mat,
                           delimiter = ',')              
                np.savetxt('NE_C14_10Be_ER0.0_expC.csv',
                           C_export,
                           delimiter = ',')
                np.savetxt('NE_C14_10Be_ER0.0_expBe.csv',
                           Be_export,
                           delimiter = ',')
                
        return C_tot, Be_tot, C_Be_surf, C_surf, Be_surf, ratios

In [ ]:
CRN_loop_fxn(total_time, time_change)